In [1]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter

In [5]:
def daily_aggregation(df_rd):
    # Compute sentiment variables aggregated over a day
    df_agg = df_rd.groupby('Date').agg(\
                                        rd_neg_sum = ('rd_neg', 'sum'),\
                                        rd_neg_mean = ('rd_neg', 'mean'),\
                                        rd_neg_std = ('rd_neg', 'std'),\
                                        rd_pos_sum = ('rd_pos', 'sum'),\
                                        rd_pos_mean = ('rd_pos', 'mean'),\
                                        rd_pos_std = ('rd_pos', 'std'),\
                                        rd_comp_sum = ('rd_comp', 'sum'),\
                                        rd_comp_mean = ('rd_comp', 'mean'),\
                                        rd_comp_std = ('rd_comp', 'std'),\
                                        rd_RelDiffPosNeg_sum = ('rd_RelDiffPosNeg', 'sum'),\
                                        rd_RelDiffPosNeg_mean = ('rd_RelDiffPosNeg', 'mean'),\
                                        rd_RelDiffPosNeg_std = ('rd_RelDiffPosNeg', 'std'),\
                                        rd_Ups_sum = ('Ups', 'sum'),\
                                        rd_Ups_mean = ('Ups', 'mean'),\
                                        rd_Ups_std = ('Ups', 'std'),\
                                        rd_Upvote_Ratio_sum = ('Upvote Ratio', 'sum'),\
                                        rd_Upvote_Ratio_mean = ('Upvote Ratio', 'mean'),\
                                        rd_Upvote_Ratio_std = ('Upvote Ratio', 'std'),\
                                        rd_Num_Comments_sum = ('Num Comments', 'sum'),\
                                        rd_Num_Comments_mean = ('Num Comments', 'mean'),\
                                        rd_Num_Comments_std = ('Num Comments', 'std'),\
                                        rd_Total_Awards_sum = ('Total Awards', 'sum'),\
                                        rd_Total_Awards_mean = ('Total Awards', 'mean'),\
                                        rd_Total_Awards_std = ('Total Awards', 'std'),\
                                      )

    # Compute the number of positive (compound>0) and negative (compound<0) rdeets per day

    df_agg['rd_comp_q5_count'] = df_rd.groupby('Date')['rd_comp_quintile'].apply(lambda x: (x==5).sum()).reset_index(name='rd_comp_q5_count')['rd_comp_q5_count'].to_list()
    df_agg['rd_comp_q4_count'] = df_rd.groupby('Date')['rd_comp_quintile'].apply(lambda x: (x==4).sum()).reset_index(name='rd_comp_q4_count')['rd_comp_q4_count'].to_list()
    df_agg['rd_comp_q3_count'] = df_rd.groupby('Date')['rd_comp_quintile'].apply(lambda x: (x==3).sum()).reset_index(name='rd_comp_q3_count')['rd_comp_q3_count'].to_list()
    df_agg['rd_comp_q2_count'] = df_rd.groupby('Date')['rd_comp_quintile'].apply(lambda x: (x==2).sum()).reset_index(name='rd_comp_q2_count')['rd_comp_q2_count'].to_list()
    df_agg['rd_comp_q1_count'] = df_rd.groupby('Date')['rd_comp_quintile'].apply(lambda x: (x==1).sum()).reset_index(name='rd_comp_q1_count')['rd_comp_q1_count'].to_list()

    # Reset index and reformat date
    df_agg = df_agg.reset_index()
    df_agg.Date = pd.to_datetime(df_agg.Date)

    # Compute the number of positive (compound>0) and negative (compound<0) rdeets per day
    df_agg['rd_N'] = df_rd.groupby('Date')['rd_comp'].apply(lambda x: x.count()).reset_index(name='rd_N')['rd_N'].to_list()
    df_agg['rd_N_pos'] = df_rd.groupby('Date')['rd_comp'].apply(lambda x: (x>0).sum()).reset_index(name='rd_N_pos')['rd_N_pos'].to_list()
    df_agg['rd_N_neg'] = df_rd.groupby('Date')['rd_comp'].apply(lambda x: (x<0).sum()).reset_index(name='rd_N_neg')['rd_N_neg'].to_list()

    # Compute most dicussed topic of the day
    df_agg['rd_topic1_freq'] = df_rd.groupby('Date')['rd_topic'].apply(lambda x: Counter(list(x))[0]/len(list(x))).reset_index(name='rd_topic1_freq')['rd_topic1_freq'].to_list()
    df_agg['rd_topic2_freq'] = df_rd.groupby('Date')['rd_topic'].apply(lambda x: Counter(list(x))[1]/len(list(x))).reset_index(name='rd_topic2_freq')['rd_topic2_freq'].to_list()
    df_agg['rd_topic3_freq'] = df_rd.groupby('Date')['rd_topic'].apply(lambda x: Counter(list(x))[2]/len(list(x))).reset_index(name='rd_topic3_freq')['rd_topic3_freq'].to_list()

    # Add influencers
    for influencer in ['AlphaGrayWolf', 'Secret_Operative', 'npjprods']:
        df_agg[influencer] = df_rd.groupby('Date')['Author'].apply(lambda x: int(influencer in list(x))).reset_index(name=influencer)[influencer].to_list()

    # Compute rolling window variables
    variables = list(df_agg.filter(like='rd_').filter(like='_sum').columns)
    variables = variables + ['rd_N', 'rd_N_pos', 'rd_N_neg']
    for var in variables:
        df_agg[var+'_last5_mean'] = df_agg[var].rolling(5).mean().reset_index(name=var+'_last5_mean')[var+'_last5_mean'].to_list()
        df_agg[var+'_last5_std'] = df_agg[var].rolling(5).std().reset_index(name=var+'_last5_std')[var+'_last5_std'].to_list()
        df_agg[var+'_last10_mean'] = df_agg[var].rolling(10).mean().reset_index(name=var+'_last10_mean')[var+'_last10_mean'].to_list()
        df_agg[var+'_last10_std'] = df_agg[var].rolling(10).std().reset_index(name=var+'_last10_std')[var+'_last10_std'].to_list()

    # Fill missing values
    for col in [c for c in df_agg.columns if c.endswith('std')]:
        df_agg[col].fillna(0, inplace=True)
        
    # Remove rows with missing values
    df_agg.dropna(inplace=True)


    return df_agg

In [6]:
# Read the raw data frame
df_rd = pd.read_csv('df_rd_topic.csv')

# Perform aggregation
df_agg = daily_aggregation(df_rd)

# Store the dataframe
df_agg.to_csv('df_rd_agg.csv', index=False)

In [7]:
df_agg

,Date,rd_neg_sum,rd_neg_mean,rd_neg_std,rd_pos_sum,rd_pos_mean,rd_pos_std,rd_comp_sum,rd_comp_mean,rd_comp_std,...,rd_N_last10_mean,rd_N_last10_std,rd_N_pos_last5_mean,rd_N_pos_last5_std,rd_N_pos_last10_mean,rd_N_pos_last10_std,rd_N_neg_last5_mean,rd_N_neg_last5_std,rd_N_neg_last10_mean,rd_N_neg_last10_std
9,2020-03-29,0.000,0.000000,0.000000,0.200,0.200000,0.000000,0.3080,0.308000,0.000000,...,1.7,0.674949,0.6,0.547723,0.7,0.674949,1.0,0.707107,1.0,0.666667
10,2020-03-30,0.052,0.017333,0.030022,0.540,0.180000,0.073750,0.9386,0.312867,0.042750,...,1.8,0.788811,1.2,1.095445,0.9,0.994429,0.8,0.836660,0.9,0.737865
11,2020-03-31,0.000,0.000000,0.000000,0.285,0.142500,0.051619,0.7495,0.374750,0.038396,...,1.9,0.737865,1.4,1.140175,1.1,0.994429,0.6,0.894427,0.8,0.788811
12,2020-04-01,0.186,0.093000,0.131522,0.322,0.161000,0.077782,0.5098,0.254900,0.179464,...,1.8,0.632456,1.8,0.836660,1.1,0.994429,0.2,0.447214,0.7,0.823273
13,2020-04-02,0.000,0.000000,0.000000,0.357,0.357000,0.000000,0.7269,0.726900,0.000000,...,1.8,0.632456,1.8,0.836660,1.1,0.994429,0.0,0.000000,0.7,0.823273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,2021-03-14,0.517,0.073857,0.095710,1.631,0.233000,0.189853,0.8941,0.127729,0.467417,...,5.1,2.558211,4.0,2.645751,3.4,1.955050,2.0,0.707107,1.7,1.251666
328,2021-03-15,0.965,0.321667,0.105983,0.000,0.000000,0.000000,-1.3245,-0.441500,0.149170,...,5.1,2.558211,3.4,3.209361,3.1,2.233582,2.4,0.547723,2.0,1.154701
329,2021-03-16,0.136,0.045333,0.078520,0.228,0.076000,0.080299,-0.0062,-0.002067,0.395543,...,4.7,2.540779,3.2,3.271085,3.0,2.260777,2.2,0.836660,1.7,0.948683
330,2021-03-17,1.282,0.142444,0.182742,1.489,0.165444,0.153921,0.7206,0.080067,0.523863,...,5.3,2.790858,4.2,3.193744,3.4,2.412928,2.4,0.894427,1.9,0.994429


# Some analytics

In [ ]:
df_rd.groupby('Date').Author.apply(lambda x: int('AlphaGrayWolf' in x))

In [ ]:
df_rd.columns

In [ ]:
AlphaGrayWolf
Secret_Operative
Random1DollarTip

In [ ]:
df_rd.groupby('Author').Ups.agg('mean').sort_values(ascending=False).head(50)

In [ ]:
df_rd.groupby('Author')['Total Awards'].agg('sum').sort_values(ascending=False).head(50)

In [ ]:
df_rd_raw.query('Author == "AlphaGrayWolf"')

In [ ]:
df_rd_raw[df_rd_raw.Author == '[deleted]']

In [ ]:
ax = df_rd_raw['Upvote Ratio'].hist(alpha=0.3)
ax.set_yscale('log')

In [ ]:
ax = df_rd_raw.Ups.hist(bins=100, alpha=0.3)
ax.set_yscale('log')

In [ ]:
ax = df_rd_raw.query('Ups > 100').Ups.hist(bins=100, alpha=0.3)
ax.set_yscale('log')

In [ ]:
ax = df_rd.rd_comp.hist(bins=100, alpha=0.3)
ax.set_yscale('log')

In [ ]:
df_rd.query('rd_comp == 0').shape[0]/df_rd.shape[0]

In [ ]:
df_rd.query('rd_comp != 0').shape[0]

In [ ]:
ax = df_rd['rd_comp'].hist(bins=100, alpha=0.3)
ax.set_yscale('log')